In [ ]:
import os, time
import matplotlib.pyplot as plt
import itertools
import pickle
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.utils as vutils
import cv2

from torchvision import datasets, transforms
from numpy import ndarray
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torch.utils.data as data
from torch.utils.data import Dataset, TensorDataset

def read_images(dataset_path):
    # Image Parameters
    image_height = 14 #CHANGE HERE, the image height to be resized to
    image_width = 14 #CHANGE HERE, the image width to be resized to

    imagepaths, labels = list(), list()

    # An ID will be affected to each sub-folders by alphabetical order
    label = 0
    # List the directory
    try:  # Python 2
        classes = sorted(os.walk(dataset_path).next()[1])
    except Exception:  # Python 3
        classes = sorted(os.walk(dataset_path).__next__()[1])

    # List each sub-directory (the classes)
    for c in classes:
        c_dir = os.path.join(dataset_path, c)

        try:  # Python 2
            walk = os.walk(c_dir).next()

        except Exception:  # Python 3
            walk = os.walk(c_dir).__next__()
        # Add each image to the training set

        for sample in walk[2]:
            # Only keeps jpeg images
            if sample.endswith('.jpg'):
                img = cv2.imread(os.path.join(c_dir, sample))
                img = cv2.resize(img,(image_height,image_width),interpolation = cv2.INTER_AREA)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                imagepaths.append(img)
                labels.append(label)

        label += 1

    print (len(imagepaths))
    print (len(labels))

    return imagepaths, labels

def read_orignalimages(dataset_path):
    # Image Parameters
    image_height = 14 #CHANGE HERE, the image height to be resized to
    image_width = 14 #CHANGE HERE, the image width to be resized to

    imagepaths, labels = list(), list()

    # An ID will be affected to each sub-folders by alphabetical order
    label = 0
    # List the directory
    try:  # Python 2
        classes = sorted(os.walk(dataset_path).next()[1])
    except Exception:  # Python 3
        classes = sorted(os.walk(dataset_path).__next__()[1])

    # List each sub-directory (the classes)
    for c in classes:
        c_dir = os.path.join(dataset_path, c)

        try:  # Python 2
            walk = os.walk(c_dir).next()

        except Exception:  # Python 3
            walk = os.walk(c_dir).__next__()
        # Add each image to the training set

        for sample in walk[2]:
            # Only keeps jpeg images
            if sample.endswith('.png'):
                img = cv2.imread(os.path.join(c_dir, sample))
                img = cv2.resize(img,(image_height,image_width),interpolation = cv2.INTER_AREA)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                imagepaths.append(img)
                labels.append(label)

        label += 1

    print (len(imagepaths))
    print (len(labels))

    return imagepaths, labels

Orignal_Path = r'' #The dataset file or root folder path. 
LL_Path = r'' #The dataset file or root folder path.
LH_Path = r'' #The dataset file or root folder path.
HL_Path = r'' #The dataset file or root folder path.
HH_Path = r'' #The dataset file or root folder path.
LL_input, LLy_input = read_images(LL_Path)
LH_input, LHy_input = read_images(LH_Path)
HL_input, HLy_input = read_images(HL_Path)
HH_input, HHy_input = read_images(HH_Path)
Orignal_input, Orignaly_input=read_orignalimages(Orignal_Path)

LL_input = np.asarray(LL_input)
LL_input = (LL_input.astype(np.float32) - LL_input.mean()) / LL_input.std()
LL_input = LL_input.astype(np.float32)
LL_input = np.expand_dims(LL_input, axis=1)
LL_input = torch.tensor(LL_input)

LH_input = np.asarray(LH_input)
LH_input = (LH_input.astype(np.float32) - LH_input.mean()) / LH_input.std()
LH_input = LH_input.astype(np.float32) 
LH_input = np.expand_dims(LH_input, axis=1)
LH_input = torch.tensor(LH_input)

HL_input = np.asarray(HL_input)
HL_input = (HL_input.astype(np.float32) - HL_input.mean()) / HL_input.std()
HL_input = HL_input.astype(np.float32) 
HL_input = np.expand_dims(HL_input, axis=1)
HL_input = torch.tensor(HL_input)

HH_input = np.asarray(HH_input)
HH_input = (HH_input.astype(np.float32) - HH_input.mean()) / HH_input.std()
HH_input = HH_input.astype(np.float32) 
HH_input = np.expand_dims(HH_input, axis=1)
HH_input = torch.tensor(HH_input)

Orignal_input = np.asarray(Orignal_input)
Orignal_input = (Orignal_input.astype(np.float32) - Orignal_input.mean()) / Orignal_input.std()
Orignal_input= Orignal_input.astype(np.float32) 
Orignal_input = np.expand_dims(Orignal_input, axis=1)
Orignal_input = torch.tensor(Orignal_input)

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]
        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]
        w = self.tensors[2][index]
        h = self.tensors[3][index]
        z = self.tensors[4][index]
      
        return x, y,w,h,z

    def __len__(self):
        return self.tensors[0].size(0)

def min_max_normalization(tensor):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (1 - 0) + 0
    return tensor

train_dataset_normal = CustomTensorDataset(tensors=(LL_input, LH_input,HL_input,HH_input,Orignal_input), transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal , batch_size=200)

# obtain one batch of training images
dataiter = iter(train_loader)
LL_input, LH_input, HL_input, HH_input, Orignal_input = dataiter.next()


LL_input = LL_input.numpy()

# get one image from the batch
img = np.squeeze(LL_input[0])
fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

LH_input = LH_input.numpy()

# get one image from the batch
img = np.squeeze(LH_input[0])
fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

HL_input = HL_input.numpy()

# get one image from the batch
img = np.squeeze(HL_input[0])
fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

HH_input = HH_input.numpy()

# get one image from the batch
img = np.squeeze(HH_input[0])
fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

Orignal_input = Orignal_input.numpy()


# get one image from the batch
img = np.squeeze(Orignal_input[0])
fig = plt.figure(figsize = (5,5)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')

class Classifier(nn.Module):
    
    def __init__(self, D_in, H1, D_out):
        super().__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.dropout1 = nn.Dropout(0.2)
        self.linear2 = nn.Linear(H1, D_out)
        self.softmax = torch.nn.Softmax(dim=1)
    def forward(self, x):
        x = F.leaky_relu(self.linear1(x)) 
        x = self.dropout1(x)
        # we need raw output in case of multiclass classification so not using any activation function
        x = self.linear2(x)
        x = self.softmax(x)
        return x
model1 = Classifier(196, 20, 10)
model2 = Classifier(196, 20, 10)
model3 = Classifier(196, 20, 10)
model4 = Classifier(196, 20, 10)
model5 = Classifier(196, 20, 10)

model1.load_state_dict(torch.load('D:\Faheem\model_save\WithDroput20\MNISTLL20%drop20softmax.pth'))
model2.load_state_dict(torch.load('D:\Faheem\model_save\WithDroput20\MNISTLH20%drop20softmax.pth'))
model3.load_state_dict(torch.load('D:\Faheem\model_save\WithDroput20\MNISTHL20%drop20softmax.pth'))
model4.load_state_dict(torch.load('D:\Faheem\model_save\WithDroput20\MNISTHH20%drop20softmax.pth'))
model5.load_state_dict(torch.load('D:\Faheem\model_save\WithDroput20\MNISTOrignal20%drop20softmax.pth'))
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
LL = ndarray((10000,10),float)
LH = ndarray((10000,10),float)
HL = ndarray((10000,10),float)
HH = ndarray((10000,10),float)
orignal = ndarray((10000,10),float)
i=0
for data in train_loader:
    #print("Length of data",len(data))
    LL_input,LH_input,HL_input,HH_input,Orignal_input = data
    LL_input = LL_input.view(LL_input.shape[0], -1)
    LH_input = LH_input.view(LH_input.shape[0], -1)
    HL_input = HL_input.view(HL_input.shape[0], -1)
    HH_input = HH_input.view(HH_input.shape[0], -1)
    Orignal_input = Orignal_input.view(Orignal_input.shape[0], -1) 
    #rint(Orignal_input.shape)
    output1 = model1(Orignal_input)
    #print(output1)
    arr=output1.data.cpu().numpy()
    #print(LL[:,:].shape)
    LL[:,:]=arr
    #print('Values in LL_Model',arr)
    np.savetxt("output1.csv",LL, delimiter=", ",header='0,1,2,3,4,5,6,7,8,9',newline='\n')
    _, pred1 = torch.max(output1, 1)
    output2 = model2(Orignal_input)
    arr1=output2.data.cpu().numpy()
    LH[:,:]=arr1
    np.savetxt("output2.csv", LH, delimiter=", ",header='0,1,2,3,4,5,6,7,8,9',newline='\n')
    _, pred2 = torch.max(output2, 1)
    output3 = model3(Orignal_input)
    arr3=output3.data.cpu().numpy()
    HL[:,:]=arr3
    np.savetxt("output3.csv", HL, delimiter=", ",header='0,1,2,3,4,5,6,7,8,9',newline='\n')
    _, pred3 = torch.max(output2, 1)
    output4 = model4(Orignal_input)
    arr4=output4.data.cpu().numpy()
    #print('Values in LH_Model',arr4)
    HH[:,:]=arr4
    _, pred4 = torch.max(output4, 1)
    np.savetxt("output4.csv", HH, delimiter=", ",header='0,1,2,3,4,5,6,7,8,9',newline='\n')
    output5 = model5(Orignal_input)
    arr5=output5.data.cpu().numpy()
    #print('Values in LH_Model',arr4)
    orignal[:,:]=arr5
    _, pred5 = torch.max(output5, 1)
    np.savetxt("output5.csv", orignal, delimiter=", ",header='0,1,2,3,4,5,6,7,8,9',newline='\n')
    #print(orignal)
    print(len(Orignal_input))
    print('Class Prediction Using LL-Band=',pred1)
    print('Class Prediction Using LH-Band=',pred2)
    print('Class Prediction Using HL-Band=',pred3)
    print('Class Prediction Using HH-Band=',pred4)
    print('Class Prediction Using Orignal_Images=',pred5)
    y=output1+output2+output3+output4
    y=y/4
    print('Average Value of Bands= \n',y)
    print('Maximum Value with class= \n',torch.max(y,1))
    i=i+1
    
total=0
correct=0
for data in train_loader:
    #print("Length of data",len(data))
        
    LL_input,LH_input,HL_input,HH_input,Orignal_input = data
    LL_input = LL_input.view(LL_input.shape[0], -1)
    LH_input = LH_input.view(LH_input.shape[0], -1)
    HL_input = HL_input.view(HL_input.shape[0], -1)
    HH_input = HH_input.view(HH_input.shape[0], -1)
    Orignal_input = Orignal_input.view(Orignal_input.shape[0], -1) 
    #rint(Orignal_input.shape)
    Orignaly_input=torch.tensor(Orignaly_input)
   # Orignaly_input=Orignaly_input.cpu().clone().detach()
    output1 = model1(Orignal_input)
    _, pred1 = torch.max(output1, 1)
    
    total += Orignaly_input.size(0)
   # print(pred1)
    correct += (pred1== Orignaly_input).sum().item()
print('Test Accuracy of the LL_Band_model on the 10000 Orignal test images: {} %'.format((correct / total) * 100))